In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
%run -i ../notebooks/functions.py

In [6]:
#get NTA centroid list
NTA = pd.read_csv('../processed_data/geocoded_population.csv').loc[:, ['latitude', 'longitude']]
print(NTA.shape)
NTA.head()

(195, 2)


,latitude,longitude
0,40.768352,-73.809546
1,40.763352,-73.868396
2,40.734894,-73.783716
3,40.755734,-73.983503
4,40.818055,-73.856188


## Quality of Life

In [12]:
#import data
library_df = pd.read_csv('../processed_data/libraries.csv')
parks_df = pd.read_csv('../processed_data/parks.csv')
hotels_df = pd.read_csv('../processed_data/hotels.csv')
facilities_df = pd.read_csv('../processed_data/facilities.csv')

In [16]:
facilities_df.head()

,factype,latitude,longitude
0,PARKING LOTS AND GARAGES,40.603390,-73.996779
1,PARKING LOTS AND GARAGES,40.712552,-73.953341
2,PARKING LOTS AND GARAGES,40.701827,-73.817315
3,PARKING LOTS AND GARAGES,40.768716,-73.957201
4,PARKING LOTS AND GARAGES,40.824592,-73.843382


In [25]:
#bin the simple dataframes
library_bin = bin_data(library_df, 'count')
parks_bin = bin_data(parks_df, 'sum')
hotels_bin = bin_data(hotels_df, 'count')

In [26]:
#check
print(hotels_bin.shape)
hotels_bin.head()

(195, 3)


,NTA_lat,NTA_long,count
0,40.754313,-73.969018,469.0
1,40.755734,-73.983503,449.0
2,40.720766,-74.004748,416.0
3,40.774847,-73.984699,353.0
4,40.777325,-73.948683,111.0


#### Facilities DF

In [106]:
#pull value counts for each category for each neighborhood
facilities_count = bin_data(facilities_df).value_counts(['factype', 'close_NTA_lat', 'close_NTA_long'])

In [143]:
#get the index corresponding to the subgroup names
factypes = facilities_count.index.get_level_values(0).unique()
#empty dict to save facility dataframes
facility_bin = {}
#column list to create new column names
fac_columns = ['NTA_lat', 'NTA_long']

#create a dataframe for each subgroup and append name to column list
for factype in factypes:
    facility_bin[factype] = pd.DataFrame(facilities_count[factype], columns=['count']).reset_index()
    fac_columns.append(factype)
    
#check how many subgroup and merge all back in onto the lat_long
print(len(facility_bin))
facilities_all = facility_bin[factypes[0]].merge(facility_bin[factypes[1]],how='outer',on=['close_NTA_lat', 
                                'close_NTA_long']).merge(facility_bin[factypes[2]],how='outer', on=['close_NTA_lat', 
                                'close_NTA_long']).merge(facility_bin[factypes[3]],how='outer', on=['close_NTA_lat', 
                                'close_NTA_long']).merge(facility_bin[factypes[4]],how='outer', on=['close_NTA_lat', 
                                'close_NTA_long']).merge(facility_bin[factypes[5]],how='outer', on=['close_NTA_lat', 
                                'close_NTA_long']).merge(facility_bin[factypes[6]],how='outer',on=['close_NTA_lat', 
                                'close_NTA_long']).merge(facility_bin[factypes[7]],how='outer', on=['close_NTA_lat', 
                                'close_NTA_long'])


facilities_all.columns = fac_columns


8


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'count_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


In [144]:
#check merged dataframe
print(facilities_all.shape)
facilities_all.head()

(193, 10)


,NTA_lat,NTA_long,PARKING LOTS AND GARAGES,NON-PUBLIC K-12 SCHOOLS,HOSPITALS AND CLINICS,PUBLIC K-12 SCHOOLS,"STREETSCAPES, PLAZAS, AND MALLS",BUS DEPOTS AND TERMINALS,MUSEUMS,COLLEGES OR UNIVERSITIES
0,40.754313,-73.969018,115.0,9.0,13.0,2,8.0,NaN,4.0,3.0
1,40.764964,-73.955088,102.0,4.0,31.0,9,2.0,NaN,2.0,4.0
2,40.755734,-73.983503,85.0,NaN,3.0,2,6.0,NaN,7.0,9.0
3,40.744221,-73.977515,85.0,5.0,19.0,8,2.0,NaN,NaN,3.0
4,40.747581,-73.999020,82.0,8.0,17.0,15,1.0,NaN,5.0,5.0


In [149]:
#add 0 for missing
#for each NTA value
for lat, long in NTA.values:
    #check that it is not in the grouped dataframe
    if lat not in facilities_all['NTA_lat'].values and long not in facilities_all['NTA_long'].values:
            #if not then append a row with that NTA lat long
            neighborhood = {'NTA_lat': lat, 'NTA_long': long}
            facilities_all = facilities_all.append(neighborhood, ignore_index=True)
            
facilities_final = facilities_all.fillna(0)

In [151]:
print(facilities_final.shape)
facilities_final.head()

(195, 10)


,NTA_lat,NTA_long,PARKING LOTS AND GARAGES,NON-PUBLIC K-12 SCHOOLS,HOSPITALS AND CLINICS,PUBLIC K-12 SCHOOLS,"STREETSCAPES, PLAZAS, AND MALLS",BUS DEPOTS AND TERMINALS,MUSEUMS,COLLEGES OR UNIVERSITIES
0,40.754313,-73.969018,115.0,9.0,13.0,2.0,8.0,0.0,4.0,3.0
1,40.764964,-73.955088,102.0,4.0,31.0,9.0,2.0,0.0,2.0,4.0
2,40.755734,-73.983503,85.0,0.0,3.0,2.0,6.0,0.0,7.0,9.0
3,40.744221,-73.977515,85.0,5.0,19.0,8.0,2.0,0.0,0.0,3.0
4,40.747581,-73.999020,82.0,8.0,17.0,15.0,1.0,0.0,5.0,5.0
